# 데이터프레임 전처리

In [15]:
import json
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

In [16]:
# 데이터 불러오기
MP = pd.read_json('MP.json', orient = 'table')
PB = pd.read_json("PB.json", orient = 'table')

In [17]:
# 국회의원 별 df
MP

,의원명,정당,소속위원회,지역,성별,당선횟수,당선방법
0,강기윤,국민의힘,[보건복지위원회],경남 창원시성산구,남,재선,지역구
1,강대식,국민의힘,"[국토교통위원회, 예산결산특별위원회]",대구 동구을,남,초선,지역구
2,강득구,더불어민주당,"[교육위원회, 국회운영위원회, 예산결산특별위원회]",경기 안양시만안구,남,초선,지역구
3,강민국,국민의힘,"[민생경제안정특별위원회, 정무위원회, 예산결산특별위원회]",경남 진주시을,남,초선,지역구
4,강민정,더불어민주당,"[중앙선거관리위원회위원(남래진)선출에관한인사청문특별위원회, 교육위원회, 국회운영...",비례대표,여,초선,비례대표
...,...,...,...,...,...,...,...
294,홍익표,더불어민주당,[문화체육관광위원회],서울 중구성동구갑,남,3선,지역구
295,홍정민,더불어민주당,"[산업통상자원중소벤처기업위원회, 여성가족위원회]",경기 고양시병,여,초선,지역구
296,황보승희,국민의힘,[문화체육관광위원회],부산 중구영도구,여,초선,지역구
297,황운하,더불어민주당,[정무위원회],대전 중구,남,초선,지역구


In [4]:
# 법률안 별 df
PB.head()

,법률안명,대표발의자,공동발의자,소관위원회,제안일,처리상태,상세페이지
0,성폭력방지 및 피해자보호 등에 관한 법률 | 일부개정법률안,강대식,"[김병욱, 김상훈, 배준영, 송언석, 유경준, 유의동, 이만희, 이명수, 조수진, ...",여성가족위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
1,남녀고용평등과 일·가정 양립 지원에 관한 법률 | 일부개정법률안,강대식,"[김병욱, 김승수, 배준영, 유경준, 유의동, 이명수, 정희용, 조수진, 지성호]",환경노동위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
2,양성평등기본법 | 일부개정법률안,강대식,"[김병욱, 김상훈, 김승수, 배준영, 송언석, 유경준, 유의동, 이만희, 이명수, ...",여성가족위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
3,여수ㆍ순천 10ㆍ19사건 진상규명 및 희생자 명예회복에 관한 특별법 | 일부개정법률안,소병철,"[김승남, 김정호, 김회재, 서동용, 이수진, 이용선, 정일영, 주철현, 한준호]",행정안전위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...
4,조세특례제한법 | 일부개정법률안,정우택,"[김학용, 서정숙, 이양수, 이종성, 정운천, 정희용, 조명희, 지성호, 최승재, ...",기획재정위원회,2022-06-30,NaN,http://likms.assembly.go.kr/bill/billDetail.do...


In [18]:
# 의원별로 새로운 df 생성
person_li = MP['의원명'].tolist() + MP['의원명'].tolist()
person_se = pd.Series(person_li)

name_pb = pd.DataFrame(person_se, columns = ['의원명'])

#이름 오름차순 정렬
name_pb = name_pb.sort_values(['의원명'])

#인덱스 다시 설정
name_pb.reset_index(inplace=True, drop=True)

name_pb

,의원명
0,강기윤
1,강기윤
2,강대식
3,강대식
4,강득구
...,...
593,황보승희
594,황운하
595,황운하
596,황희


In [19]:
type(name_pb)

pandas.core.frame.DataFrame

In [20]:
# 발의자 설정(대표/ 공동)
name_pb['발의자'] = '대표'
name_pb.iloc[1::2,1] = '공동'

# 법률안명 추가
name_pb['법률안명'] = ''
name_pb['소관위원회'] = ''

In [21]:
# 발의자가 대표인 경우
for i in list(PB['대표발의자']): #법률안 df 에서 의원명
    
    li = name_pb[(name_pb['의원명']==i)&(name_pb['발의자'] == '대표')]['법률안명'].tolist() #리스트로 변경
    j = name_pb.index[(name_pb['의원명']==i)&(name_pb['발의자'] == '대표')]
    j = j.tolist() #int64타입이라 리스트로 변경
    
    for t in list(PB[PB['대표발의자']==i]['법률안명']): #해당 법률안 리스트로 append
        li.append(t)
        
    name_pb.iloc[j[0],2] = str(li) #li 값 추가
        
name_pb

IndexError: list index out of range

In [36]:
li = name_pb[(name_pb['의원명']=='강대식')&(name_pb['발의자'] == '대표')]['법률안명'].tolist() 

In [37]:
j = name_pb.index[(name_pb['의원명']=='강대식')&(name_pb['발의자'] == '대표')]

In [38]:
j = j.tolist()

In [39]:
for i in list(PB[PB['대표발의자']=='강대식']['법률안명']):
    li.append(i)
    
li

['[\'[\\\'[\\\\\\\'[\\\\\\\\\\\\\\\'[\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\'[\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\'[\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\'["[\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\'\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\', \\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

In [40]:
name_pb.iloc[j[0],2] = str(li)

In [41]:
name_pb.iloc[j[0],2]

'[\'[\\\'[\\\\\\\'[\\\\\\\\\\\\\\\'[\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\'[\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\'[\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\'[\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\'["[\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\

In [42]:
name_pb.iloc[j[0],:]

의원명                                                    강대식
발의자                                                     대표
법률안명     ['[\'[\\\'[\\\\\\\'[\\\\\\\\\\\\\\\'[\\\\\\\\\...
소관위원회                                                     
Name: 2, dtype: object